## Get Absence Data

### after discussion with lecturer, it was decided not to generate psuedo-absense data. Datasets were loaded again to provide legitimate absence data.

### Import packages

In [1]:
import numpy as np
import math
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
sns.set_style('whitegrid')
import glob
import folium
import dask.dataframe as dd
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print('imports ok')

imports ok


### the existing presence data was used in ArcGIS software to determine the geographical boundaries of the required absence data.  


In [47]:
obs = '../Datasets/selected_observations.csv'   # read presence data
df1 = pd.read_csv(obs)
print(obs + " read")
df1.head

../Datasets/selected_observations.csv read


<bound method NDFrame.head of       OBJECTID  Field1 SAMPLING_EVENT_ID    LOC_ID   LATITUDE  LONGITUDE  \
0            1     148         S31224100   L285222  43.659986 -79.309355   
1            2     297         S31794143   L285222  43.659986 -79.309355   
2            3     306         S32061821   L285222  43.659986 -79.309355   
3            4     347         S30776835   L285222  43.659986 -79.309355   
4           24    4861         S15020778  L2334475  42.582348 -80.433226   
5           31    8221         S12221651   L248862  45.744946 -84.899468   
6           32    8877         S33287032   L355449  46.746349 -92.147533   
7           39   11185          S8526583   L131153  41.955399 -82.514000   
8          118   28836         S32036860  L4986825  46.640196 -86.215532   
9          126   33061         S22867147   L129756  42.158680 -80.099457   
10         140   39611         S31959637   L935436  42.561426 -80.250320   
11         177   47604         S10785184  L1924940  41.752

### get geographical boundaries

In [48]:
minlat = df1.LATITUDE.min()
maxlat = df1.LATITUDE.max()

minlong = df1.LONGITUDE.min()
maxlong = df1.LONGITUDE.max()

print("minlat: ",  minlat)
print("maxlat: ",  maxlat)
print("minlong: ",  minlong)
print("maxlong: ",  maxlong)

minlat:  41.4759174
maxlat:  46.78324
minlong:  -92.147533
maxlong:  -76.19546409999998


### Load the bird observation data 

<ul>
<li>glob used for reading multiple files
<li>chunksize used to manage memory usage
<li>condition statement to extract records within the required geographical range
<li>saved to absence.csv
</ul>

In [50]:


## filenames = 'ERD2016SS/reference/2016/checklists-sample.csv' # sample file for testing

filenames = glob.glob('../ERD2016SS/*/checklists.csv')
##filenames =  glob.glob('../ERD2016SS/2003/checklists.csv')
output = '../Datasets/absence.csv'

chunk = pd.DataFrame()
chunksize = 10**5
head = 0
with open(output, 'a') as outfile:
    for file in filenames:
        i=0
        for f in pd.read_csv(file, chunksize=chunksize, usecols=['SAMPLING_EVENT_ID','LOC_ID','LATITUDE','LONGITUDE','YEAR','MONTH','DAY','TIME','COUNTRY','STATE_PROVINCE','COUNTY','COUNT_TYPE','EFFORT_HRS','EFFORT_DISTANCE_KM','EFFORT_AREA_HA','OBSERVER_ID','NUMBER_OBSERVERS','GROUP_ID','PRIMARY_CHECKLIST_FLAG','Charadrius_melodus']):
            chunk = chunk.append(f.loc[(f.LATITUDE >= minlat) & (f.LATITUDE <= maxlat) & (f.LONGITUDE >= minlong) & (f.LONGITUDE <= maxlong)])
            i+=1
        if head == 0:        
            chunk.to_csv(outfile)
            head =+1
        else:
            chunk.to_csv(outfile, header=None)
        print(file + " has been imported.", " Chunks: ", i, " Shape: ", chunk.shape)
        chunk = pd.DataFrame()
print("done")




C:\Users\Wayne\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (18,849) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Wayne\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (13,18,849) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


../ERD2016SS\2002\checklists.csv has been imported.  Chunks:  5  Shape:  (65084, 20)
../ERD2016SS\2003\checklists.csv has been imported.  Chunks:  1  Shape:  (9805, 20)
../ERD2016SS\2004\checklists.csv has been imported.  Chunks:  1  Shape:  (10101, 20)
../ERD2016SS\2005\checklists.csv has been imported.  Chunks:  1  Shape:  (13398, 20)
../ERD2016SS\2006\checklists.csv has been imported.  Chunks:  2  Shape:  (22741, 20)
../ERD2016SS\2007\checklists.csv has been imported.  Chunks:  2  Shape:  (33134, 20)
../ERD2016SS\2008\checklists.csv has been imported.  Chunks:  3  Shape:  (40955, 20)
../ERD2016SS\2009\checklists.csv has been imported.  Chunks:  5  Shape:  (67474, 20)
../ERD2016SS\2010\checklists.csv has been imported.  Chunks:  7  Shape:  (92058, 20)


C:\Users\Wayne\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (849) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


../ERD2016SS\2011\checklists.csv has been imported.  Chunks:  10  Shape:  (123537, 20)
../ERD2016SS\2012\checklists.csv has been imported.  Chunks:  14  Shape:  (204519, 20)
../ERD2016SS\2013\checklists.csv has been imported.  Chunks:  18  Shape:  (274062, 20)
../ERD2016SS\2014\checklists.csv has been imported.  Chunks:  23  Shape:  (324616, 20)
../ERD2016SS\2015\checklists.csv has been imported.  Chunks:  29  Shape:  (395452, 20)
../ERD2016SS\2016\checklists.csv has been imported.  Chunks:  34  Shape:  (474851, 20)
done


### the following snapshot from ArcGIS shows the dataset bounded by the required geographical coordinates

<img src="..\cap2.jpg"> 

### verified that the new file contains the correct data

In [51]:

ab = '../Datasets/absence.csv'
df1 = pd.DataFrame()
df1 = pd.read_csv(ab, encoding="ISO-8859-1")
print(ab + " read")
minlat = df1.LATITUDE.min()
maxlat = df1.LATITUDE.max()
minlong = df1.LONGITUDE.min()
maxlong = df1.LONGITUDE.max()
print("minlat: ",  minlat)
print("maxlat: ",  maxlat)
print("minlong: ",  minlong)
print("maxlong: ",  maxlong)
outfile.close()
df1.head
 

C:\Users\Wayne\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (14,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


../Datasets/absence.csv read
minlat:  41.4759174
maxlat:  46.78324
minlong:  -92.147533
maxlong:  -76.1954641


<bound method NDFrame.head of          Unnamed: 0 SAMPLING_EVENT_ID    LOC_ID   LATITUDE  LONGITUDE  YEAR  \
0                31         S17168749   L152688  41.834606 -88.373852  1900   
1                32         S17203080   L152688  41.834606 -88.373852  1900   
2                33         S17203251   L152688  41.834606 -88.373852  1900   
3                43         S31296472  L4685937  42.303087 -83.316493  1900   
4                55          S9695194  L1397004  41.735986 -80.764458  1904   
5                57          S6081513   L213075  41.819331 -88.066642  1905   
6                99         S26857777       L11  43.644647 -79.462538  1921   
7               100         S26857819   L666769  43.695866 -79.384847  1921   
8               101         S26857908       L11  43.644647 -79.462538  1921   
9               102         S26859667       L11  43.644647 -79.462538  1921   
10              103         S26859715   L666769  43.695866 -79.384847  1921   
11              104   

### remove duplicated observations done by birders working in groups

In [52]:

print('Shape of DataFrame:    ', df1.shape)
df1 = df1[df1.PRIMARY_CHECKLIST_FLAG != '?'] # '?' flag identifies duplicate records
print('Shape of DataFrame:    ', df1.shape)


Shape of DataFrame:     (2151787, 21)
Shape of DataFrame:     (1895461, 21)


### check that sampling event IDs are all unique


In [53]:

temp = pd.DataFrame()
temp['is_duplicated'] = df1.duplicated('SAMPLING_EVENT_ID') # checking for duplication of sampling event ID
print('duplicate keys: ', temp['is_duplicated'].sum())



duplicate keys:  0


In [54]:
print(df1.head)

<bound method NDFrame.head of          Unnamed: 0 SAMPLING_EVENT_ID    LOC_ID   LATITUDE  LONGITUDE  YEAR  \
0                31         S17168749   L152688  41.834606 -88.373852  1900   
1                32         S17203080   L152688  41.834606 -88.373852  1900   
2                33         S17203251   L152688  41.834606 -88.373852  1900   
3                43         S31296472  L4685937  42.303087 -83.316493  1900   
4                55          S9695194  L1397004  41.735986 -80.764458  1904   
5                57          S6081513   L213075  41.819331 -88.066642  1905   
6                99         S26857777       L11  43.644647 -79.462538  1921   
7               100         S26857819   L666769  43.695866 -79.384847  1921   
8               101         S26857908       L11  43.644647 -79.462538  1921   
12              105         S27941860   L285222  43.659986 -79.309355  1921   
13              106         S27941937   L285222  43.659986 -79.309355  1921   
14              107   